In [12]:
# poppler 
!apt-get install poppler-utils 
!apt-get update
!apt-get install build-essential libpoppler-cpp-dev pkg-config python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 0s (1,943 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [18]:
# pdftotext 
!pip install pdftotext

  Using cached https://files.pythonhosted.org/packages/58/38/f04c252f4cb2d10af9abcde0a2db1bcd38288a76a99e88333f2a434e0c40/pdftotext-2.1.4.tar.gz
  Created wheel for pdftotext: filename=pdftotext-2.1.4-cp36-cp36m-linux_x86_64.whl size=54072 sha256=595227a00a518121c01b7594244219a80a12bef35ee2f7686f36862e64019b52
  Stored in directory: /root/.cache/pip/wheels/88/c3/62/c1ef7f738f8c0c1ac67cbe63786cae734d555c5077088036f0
Successfully built pdftotext


In [20]:
######### 네이버 채권 분석 

######### Import #########

# Crawling
from bs4 import BeautifulSoup
import requests
import re
import copy

# PDF library
import pdftotext 

# CSV
import csv

# to Create Folder
import os

# Save File
# from URL
from urllib.request import urlretrieve



######### Main #########

# Initialize

# Create Folder 
# Analysis      # 
# Analysis/Text # 
try:
    if not (os.path.isdir("D:\Analysis")):
        os.makedirs(os.path.join("D:\Analysis"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
try:
    if not (os.path.isdir("D:\Analysis/Text")):
        os.makedirs(os.path.join("D:\Analysis/Text"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
        
        

# Data Crawling
        
page_idx = 0 # Page Index 
pdf_idx = 0 # PDF File Index
error_count = 0 #Check FXXXXXX Error
day_list_check = []

while True:
    
    # Page Index 
    page_idx += 1
    # PDF File Index
    pdf_idx += 1
    
    # Get new Data List
    try:
        URL = "https://finance.naver.com/research/debenture_list.nhn?&page={}".format(page_idx)
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, "html.parser") 
    # End of List
    except:
        break
    
    # 
    day_list=[] # date of PDF file
    pdf_list=[] # url of PDF file
    
    # Get Date of Data
    dates = soup.select('div.box_type_m td.date')
    days = re.findall("[0-9][0-9].[0-9][0-9].[0-9][0-9]",str(dates))
    for day in days:
        # remove '.' 
        day_list.append(day.replace(".",""))
        
    # to End Squence
    # further iterates do same work
    # check Data if its same to previous one
    ### maybe : if exceed maxium page number, return last page
    ### maybe : if exceed maxium page number, return None
    if day_list == day_list_check: #why 'is' doesn't work ?
        break
    else:
        day_list_check = copy.deepcopy(day_list)
    
    # Check Process
    ### Erase later
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page :",page_idx,">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(">>> from" + day_list[0] + "~~~")
    
    # Check Duration
    # Too recent
    if all(int(day) > 180101 for day in day_list):
        print(">>> new : PASS")
        continue
    # Too past
    if all(int(day) < 40401 for day in day_list):
        print(">>> old : PASS")
        break
    
    # Get URL of Data
    files = soup.find_all(class_='file')
    pdfs = re.findall("(?<=a href=\")[^ ]+pdf",str(files))
    for pdf in pdfs:
        pdf_list.append(pdf)
    
    # another Methodology
    # to set file to 'UTF-8' format
    """setting = "# -*- encoding: utf-8 -*-"
    with open(path,'w',newline='') as data_csv:
                    write_data = csv.writer(data_csv) 
                    write_data.writerow(setting)"""
    
    # Get Proper Data
    ### Check Date
    for idx, pdf in enumerate(pdf_list):
        if 40331 < int(day_list[idx]) < 180101:
            
            # from URL
            # save PDF as file
            urlretrieve(pdf, 'D:\Analysis/Analysis.pdf')
            
            # PDF to CSV
            # Read PDF File    
            with open('D:\Analysis/Analysis.pdf','rb') as file:
            #file = open('./Analysis/Analysis.pdf','rb')
                try:
                  fileReader = pdftotext.PDF(file)
                except:
                  continue

                # Set Path
                path_CSV = 'D:\Analysis/AnalysisCSV{}.csv'.format(pdf_idx)
                path_TXT = 'D:\Analysis/Text/AnalysisTXT{}.txt'.format(pdf_idx)
                
                # Read by Page
                for pn in range(len(fileReader)):

                    # Check Process
                    ### Erase later
                    print("Page:",page_idx,"Date:",day_list[idx],"File:",idx,"Page:",pn)

                    # Error : poppler error creating page
                    # wtf
                    try:
                        # Get one Page of PDF
                        tmp = fileReader[pn]

                        # Do something
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        
                    # Error Check
                    except Exception as e:
                        error_count += 1
                        print(e,"error_count :",error_count)
                        pass
                        
                    

                    #######################################################
                    # 위의껄로 하면 에러 수는 적어짐
                    # 아래껄로 하면 파일 읽을 때 주의해야 함
                    #######################################################
                    
                    # 위 #
                    # -1 : Kernel ???
                    # 'utf-8' : Set Format
                    # without this
                    # Error : 'cp949' codec can't encode character '\u201a' in position 1184: illegal multibyte sequence
                    """   
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path,'a',-1,'utf-8',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Write a Page to CSV
                        write_data_CSV.writerow(tmp)
                        
                    # open TXT file as 'Append' mode
                    # to insert new Data
                    with open(path_TXT,'a',-1,'utf-8',newline='') as data_txt:

                        # Write a Page to TXT
                        data_txt.write(tmp)
                    """
                    
                    # 아래 #
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path_CSV,'a',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Prepare to Error
                        # in specific pdf
                        # Error : poppler error creating page
                        # wtf
                        try:
                            # Write a Page to CSV
                            write_data_CSV.writerow(tmp)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass
                    
                    with open(path_TXT,'a',newline='') as data_txt:

                        # Prepare to Error
                        try:
                            # Write a Page to TXT
                            data_txt.write(tmp)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Page: 85 File: 28 Page: 18
Page: 85 File: 28 Page: 19
Page: 85 File: 28 Page: 20
Page: 85 File: 28 Page: 21
Page: 85 File: 28 Page: 22
Page: 85 File: 28 Page: 23
Page: 85 File: 28 Page: 24
Page: 85 File: 28 Page: 25
Page: 85 File: 28 Page: 26
Page: 85 File: 28 Page: 27
Page: 85 File: 28 Page: 28
Page: 85 File: 29 Page: 0
Page: 85 File: 29 Page: 1
Page: 85 File: 29 Page: 2
Page: 85 File: 29 Page: 3
Page: 85 File: 29 Page: 4
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 86 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from120730~~~
Page: 86 File: 0 Page: 0
Page: 86 File: 0 Page: 1
Page: 86 File: 0 Page: 2
Page: 86 File: 0 Page: 3
Page: 86 File: 0 Page: 4
Page: 86 File: 0 Page: 5
Page: 86 File: 1 Page: 0
Page: 86 File: 1 Page: 1
Page: 86 File: 1 Page: 2
Page: 86 File: 1 Page: 3
Page: 86 File: 1 Page: 4
Page: 86 File: 1 Page: 5
Page: 86 File: 1 Page: 6
Page: 86 File: 1 Page: 7
Page: 86 File: 1 Page: 8
Page: 86 File: 1 Page: 9
Page: 86 File: 1 Page: 10
Page: 86 File: 2 Page

Error: ignored

In [22]:
######### 네이버 채권 분석 

######### Import #########

# Crawling
from bs4 import BeautifulSoup
import requests
import re
import copy

# PDF library
import pdftotext 

# CSV
import csv

# to Create Folder
import os

# Save File
# from URL
from urllib.request import urlretrieve



######### Main #########

# Initialize

# Create Folder 
# Analysis      # 
# Analysis/Text # 
try:
    if not (os.path.isdir("D:\Analysis")):
        os.makedirs(os.path.join("D:\Analysis"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
try:
    if not (os.path.isdir("D:\Analysis/Text")):
        os.makedirs(os.path.join("D:\Analysis/Text"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
        
        

# Data Crawling
        
page_idx = 103 # Page Index 
pdf_idx = 103 # PDF File Index
#error_count = 0 #Check FXXXXXX Error
day_list_check = []

while True:
    
    # Page Index 
    page_idx += 1
    # PDF File Index
    pdf_idx += 1
    
    # Get new Data List
    try:
        URL = "https://finance.naver.com/research/debenture_list.nhn?&page={}".format(page_idx)
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, "html.parser") 
    # End of List
    except:
        break
    
    # 
    day_list=[] # date of PDF file
    pdf_list=[] # url of PDF file
    
    # Get Date of Data
    dates = soup.select('div.box_type_m td.date')
    days = re.findall("[0-9][0-9].[0-9][0-9].[0-9][0-9]",str(dates))
    for day in days:
        # remove '.' 
        day_list.append(day.replace(".",""))
        
    # to End Squence
    # further iterates do same work
    # check Data if its same to previous one
    ### maybe : if exceed maxium page number, return last page
    ### maybe : if exceed maxium page number, return None
    if day_list == day_list_check: #why 'is' doesn't work ?
        break
    else:
        day_list_check = copy.deepcopy(day_list)
    
    # Check Process
    ### Erase later
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page :",page_idx,">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(">>> from" + day_list[0] + "~~~")
    
    # Check Duration
    # Too recent
    if all(int(day) > 180101 for day in day_list):
        print(">>> new : PASS")
        continue
    # Too past
    if all(int(day) < 40401 for day in day_list):
        print(">>> old : PASS")
        break
    
    # Get URL of Data
    files = soup.find_all(class_='file')
    pdfs = re.findall("(?<=a href=\")[^ ]+pdf",str(files))
    for pdf in pdfs:
        pdf_list.append(pdf)
    
    # another Methodology
    # to set file to 'UTF-8' format
    """setting = "# -*- encoding: utf-8 -*-"
    with open(path,'w',newline='') as data_csv:
                    write_data = csv.writer(data_csv) 
                    write_data.writerow(setting)"""
    
    # Get Proper Data
    ### Check Date
    for idx, pdf in enumerate(pdf_list):
        if 40331 < int(day_list[idx]) < 180101:
            
            # from URL
            # save PDF as file
            urlretrieve(pdf, 'D:\Analysis/Analysis.pdf')
            
            # PDF to CSV
            # Read PDF File    
            with open('D:\Analysis/Analysis.pdf','rb') as file:
            #file = open('./Analysis/Analysis.pdf','rb')
                try:
                  fileReader = pdftotext.PDF(file)
                except:
                  continue

                # Set Path
                path_CSV = 'D:\Analysis/AnalysisCSV{}.csv'.format(pdf_idx)
                path_TXT = 'D:\Analysis/Text/AnalysisTXT{}.txt'.format(pdf_idx)
                
                # Read by Page
                for pn in range(len(fileReader)):

                    # Check Process
                    ### Erase later
                    print("Page:",page_idx,"Date:",day_list[idx],"File:",idx,"Page:",pn)

                    # Error : poppler error creating page
                    # wtf
                    try:
                        # Get one Page of PDF
                        tmp = fileReader[pn]

                        # Do something
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        
                    # Error Check
                    except Exception as e:
                        error_count += 1
                        print(e,"error_count :",error_count)
                        pass
                        
                    

                    #######################################################
                    # 위의껄로 하면 에러 수는 적어짐
                    # 아래껄로 하면 파일 읽을 때 주의해야 함
                    #######################################################
                    
                    # 위 #
                    # -1 : Kernel ???
                    # 'utf-8' : Set Format
                    # without this
                    # Error : 'cp949' codec can't encode character '\u201a' in position 1184: illegal multibyte sequence
                    """   
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path,'a',-1,'utf-8',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Write a Page to CSV
                        write_data_CSV.writerow(tmp)
                        
                    # open TXT file as 'Append' mode
                    # to insert new Data
                    with open(path_TXT,'a',-1,'utf-8',newline='') as data_txt:

                        # Write a Page to TXT
                        data_txt.write(tmp)
                    """
                    
                    # 아래 #
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path_CSV,'a',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Prepare to Error
                        # in specific pdf
                        # Error : poppler error creating page
                        # wtf
                        try:
                            # Write a Page to CSV
                            write_data_CSV.writerow(tmp)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass
                    
                    with open(path_TXT,'a',newline='') as data_txt:

                        # Prepare to Error
                        try:
                            # Write a Page to TXT
                            data_txt.write(tmp)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 104 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from100510~~~
Page: 104 Date: 100510 File: 0 Page: 0
Page: 104 Date: 100510 File: 0 Page: 1
Page: 104 Date: 100510 File: 0 Page: 2
Page: 104 Date: 100510 File: 0 Page: 3
Page: 104 Date: 100510 File: 0 Page: 4
Page: 104 Date: 100510 File: 0 Page: 5
Page: 104 Date: 100510 File: 0 Page: 6
Page: 104 Date: 100510 File: 0 Page: 7
Page: 104 Date: 100510 File: 0 Page: 8
Page: 104 Date: 100510 File: 0 Page: 9
Page: 104 Date: 100510 File: 0 Page: 10
Page: 104 Date: 100510 File: 0 Page: 11
Page: 104 Date: 100503 File: 1 Page: 0
Page: 104 Date: 100503 File: 1 Page: 1
Page: 104 Date: 100503 File: 1 Page: 2
Page: 104 Date: 100503 File: 1 Page: 3
Page: 104 Date: 100503 File: 1 Page: 4
Page: 104 Date: 100503 File: 1 Page: 5
Page: 104 Date: 100503 File: 1 Page: 6
Page: 104 Date: 100503 File: 1 Page: 7
Page: 104 Date: 100503 File: 1 Page: 8
Page: 104 Date: 100503 File: 1 Page: 9
Page: 104 Date: 100503 File: 1 Page: 10
Page: 104

# 새 섹션

In [24]:
######### B.O.K 금통위 의사록 

######### Import #########

# Crawling
from bs4 import BeautifulSoup
import requests
import re
import copy

# PDF library
import pdftotext 

# CSV
import csv

# to Create Folder
import os

# Save File
# from URL
from urllib.request import urlretrieve



######### Main #########



# Initialize
# Create Folder
try:
    if not (os.path.isdir("D:\BoK")):
        os.makedirs(os.path.join("D:\BoK"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
try:
    if not (os.path.isdir("D:\BoK/Text")):
        os.makedirs(os.path.join("D:\BoK/Text"))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("폴더 생성 실패")
        exit()
        
        
# Data Crawling
        
page_idx = 0 # Page Index 
pdf_idx = 0 # PDF File Index
error_count = 0 # Check FXXXXXX Error
day_list_check = []

while True:
    
    # Page Index 
    page_idx += 1
    # PDF File Index
    pdf_idx += 1
    
    
    
    # Get new Data List
    try:
        URL = "http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}".format(page_idx)
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, "html.parser") 
    # End of List
    except:
        break
    
    # 
    day_list=[]
    pdf_list=[]
    
    # Get Date of Data
    dates = soup.select('span.titlesub')
    days = re.findall("[0-9][0-9][0-9][0-9].[0-9]+.[0-9]+",str(dates))
    
    for day in days:
        # Fill 0 to month
        if day[6] == '.':
            day = day[:5]+"0"+day[5:]
        # Fill 0 to day
        try:
            day[9]
        except:
            day = day[:7]+"0"+day[8]
        # remove '.' 
        day_list.append(day.replace(".",""))
        
    # to End Squence
    # further iterates do same work
    # check Data if its same to previous one
    ### maybe : if exceed maxium page number, return last page
    ### maybe : if exceed maxium page number, return None
    if day_list == day_list_check: #why 'is' doesn't work ?
        break
    else:
        day_list_check = copy.deepcopy(day_list)
    
    # Check Process
    ### Erase later
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page :",page_idx,">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(">>> from" + day_list[0] + "~~~")
    
    # Check Duration
    # Too recent
    if all(int(day) > 20180101 for day in day_list):
        print(">>> new : PASS")
        continue
    # Too past
    if any(int(day) < 20040401 for day in day_list):
        print(">>> old : PASS")
        break
    
    # Get URL of Data
    #files = soup.find_all('div .fileGoupBox')
    files = soup.find_all(class_='fileGoupBox')
    
    pdfs = re.findall("(?<=a href=\")[^ ]+(?=\")",str(files))
    name = re.findall("(?<=title=\").+\.[a-z][a-z][a-z]",str(files))
    
    # Get PDF only
    for idx in range(len(pdfs)):
        if (name[idx][-3:] == "pdf"):
            # Get URL of Data
            # Revise URL
            # 아니 저게 왜 들어가있는거지
            pdf_list.append(pdfs[idx].replace("amp;",""))
    
    # another Methodology
    # to set file to 'UTF-8' format
    """setting = "# -*- encoding: utf-8 -*-"
    with open(path,'w',newline='') as data_csv:
                    write_data = csv.writer(data_csv) 
                    write_data.writerow(setting)"""
    
    
    # Get Proper Data
    ### Check Date
    for idx, pdf in enumerate(pdf_list):
        if 20040331 < int(day_list[idx]) < 20180101:
            
            URL="http://www.bok.or.kr"
            # from URL
            # save PDF as file
            urlretrieve(URL+pdf, 'D:\BoK/Analysis.pdf')
            
            # PDF to CSV
            # Read PDF File    
            with open('D:\BoK/Analysis.pdf','rb') as file:
            #file = open('./BoK/Analysis.pdf','rb')
                try:
                  fileReader = pdftotext.PDF(file)
                except:
                  continue

                # Set Path
                path_CSV = 'D:\BoK/AnalysisCSV{}.csv'.format(pdf_idx)
                path_TXT = 'D:\BoK/Text/AnalysisTXT{}.txt'.format(pdf_idx)
                
                # Read by Page
                for pn in range(len(fileReader)):

                    # Check Process
                    ### Erase later
                    print("Page:",page_idx,"Date:",day_list[idx],"File:",idx,"Page:",pn)

                    # Error : poppler error creating page
                    # wtf
                    try:
                        # Get one Page of PDF
                        tmp = fileReader[pn]

                        # Do something
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        tmp=tmp.replace("  "," ")
                        tmp=tmp.replace("\t\t","\t")
                        tmp=tmp.replace("\n\n","\n")
                        
                    # Error Check
                    except Exception as e:
                        error_count += 1
                        print(e,"error_count :",error_count)
                        pass
                        
                    

                    #######################################################
                    # 위의껄로 하면 에러 수는 적어짐
                    # 아래껄로 하면 파일 읽을 때 주의해야 함
                    #######################################################
                    
                    # 위 #
                    # -1 : Kernel ???
                    # 'utf-8' : Set Format
                    # without this
                    # Error : 'cp949' codec can't encode character '\u201a' in position 1184: illegal multibyte sequence
                    """   
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path,'a',-1,'utf-8',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Write a Page to CSV
                        write_data_CSV.writerow(tmp)
                        
                    # open TXT file as 'Append' mode
                    # to insert new Data
                    with open(path_TXT,'a',-1,'utf-8',newline='') as data_txt:

                        # Write a Page to TXT
                        data_txt.write(tmp)
                    """
                    
                    # 아래 #
                    # open CSV file as 'Append' mode
                    # to insert new Data
                    with open(path_CSV,'a',newline='') as data_csv:

                        # Make Writer
                        write_data_CSV = csv.writer(data_csv) 

                        # Prepare to Error
                        # in specific pdf
                        # Error : poppler error creating page
                        # wtf
                        try:
                            # Write a Page to CSV
                            write_data_CSV.writerow(tmp)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass
                    
                    with open(path_TXT,'a',newline='') as data_txt:

                        # Prepare to Error
                        try:
                            # Write a Page to TXT
                            data_txt.write(tmp)
                        # Error Check
                        except Exception as e:
                            error_count += 1
                            print(e,"error_count :",error_count)
                            pass
                    
                    

# PDF 링크
# http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000018397&fileSn=2


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 1 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20200624~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 2 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20200117~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 3 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20190531~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 4 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20181206~~~
>>> new : PASS
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< Page : 5 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>> from20180426~~~
Page: 5 Date: 20171228 File: 7 Page: 0
Page: 5 Date: 20171228 File: 7 Page: 1
Page: 5 Date: 20171228 File: 7 Page: 2
Page: 5 Date: 20171214 File: 8 Page: 0
Page: 5 Date: 20171214 File: 8 Page: 1
Page: 5 Date: 20171214 File: 8 Page: 2
Page: 5 Date: 20171214 File: 8 Page: 3
Page: 5 Date: 20171214 File: 8 Page: 4
Page: 5 Date: 20171214 File: 8 Page: 5
Page: 5 Date: 20171207 File: 9 Page: 0
Page: 5 Date: 20171207 File: 9 Page: 1
Page: 5 Date: 20171207 File: 9 Page: 2
Page: 5 Date: 201

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [25]:
print("end?")

end?
